# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [151]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [152]:

vacation = pd.read_csv("../output_data/world_weather.csv")
vacation_df = pd.DataFrame(vacation)
vacation_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hilo,19.73,-155.09,75.20,73,20,3.36,US,1589005196
1,1,chuy,-33.70,-53.46,49.19,83,0,6.31,UY,1589005223
2,2,busselton,-33.65,115.33,66.00,39,0,4.88,AU,1589005192
3,3,rikitea,-23.12,-134.97,76.06,68,71,5.70,PF,1589005177
4,4,lata,40.16,-8.33,57.99,88,99,1.99,PT,1589005470


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [153]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [154]:
# Use Lat and Lng as locations and Humidity as the weight
locations = vacation_df[["Lat", "Lng"]]
weight = vacation_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=weight, 
    dissipating=False, 
    max_intensity=10,
    point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [155]:
# vacation_df["Max Temp"] = vacation_df["Max Temp"].astype(float)
best_weather_df = vacation_df.loc[(vacation_df["Max Temp"] >= 72) & (vacation_df["Max Temp"] <= 80) & (vacation_df["Cloudiness"] == 0) & (vacation_df["Wind Speed"] <= 10)]
best_weather_df.head(15)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,65,hami,42.80,93.45,78.85,15,0,8.88,CN,1589005482
189,214,san patricio,19.22,-104.70,75.63,66,0,3.13,MX,1589005188
211,239,namibe,-15.20,12.15,79.39,48,0,4.63,AO,1589005223
252,283,oktyabrskiy,54.48,53.47,73.27,39,0,9.33,RU,1589005538
259,292,kashan,33.98,51.44,79.61,20,0,5.99,IR,1589005541
318,354,urumqi,43.80,87.60,78.80,9,0,8.95,CN,1589005555
332,369,ixtapa,20.70,-105.20,72.41,69,0,2.17,MX,1589005217
384,423,coahuayana,18.73,-103.68,74.43,70,0,2.80,MX,1589005279
413,453,abatskoye,56.29,70.46,77.58,27,0,9.04,RU,1589005577
432,472,sumbe,-11.21,13.84,78.30,62,0,4.16,AO,1589005581


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [156]:
# Copy best_weather_df into new variable hotel_df
hotel_df = best_weather_df.copy()

# Create new column
hotel_df["Hotel Name"] = ""

hotel_df.head(15)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
61,65,hami,42.80,93.45,78.85,15,0,8.88,CN,1589005482,
189,214,san patricio,19.22,-104.70,75.63,66,0,3.13,MX,1589005188,
211,239,namibe,-15.20,12.15,79.39,48,0,4.63,AO,1589005223,
252,283,oktyabrskiy,54.48,53.47,73.27,39,0,9.33,RU,1589005538,
259,292,kashan,33.98,51.44,79.61,20,0,5.99,IR,1589005541,
318,354,urumqi,43.80,87.60,78.80,9,0,8.95,CN,1589005555,
332,369,ixtapa,20.70,-105.20,72.41,69,0,2.17,MX,1589005217,
384,423,coahuayana,18.73,-103.68,74.43,70,0,2.80,MX,1589005279,
413,453,abatskoye,56.29,70.46,77.58,27,0,9.04,RU,1589005577,
432,472,sumbe,-11.21,13.84,78.30,62,0,4.16,AO,1589005581,


In [161]:
target_search = "Hotel"
target_radius = 5000
target_type = "Hotel"

# Set up parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}


for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url,params).json()
    
    hotel_df.loc[index,"Hotel Name"] = response["results"][0]["name"]

#     print(response["results"][0]["name"])
    
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
61,65,hami,42.80,93.45,78.85,15,0,8.88,CN,1589005482,哈密环球大酒店
189,214,san patricio,19.22,-104.70,75.63,66,0,3.13,MX,1589005188,Hotel Barra de Navidad
211,239,namibe,-15.20,12.15,79.39,48,0,4.63,AO,1589005223,Hotel ibis Styles Iu Namibe
252,283,oktyabrskiy,54.48,53.47,73.27,39,0,9.33,RU,1589005538,Hotel Devon
259,292,kashan,33.98,51.44,79.61,20,0,5.99,IR,1589005541,Atlasi House Hotel


In [162]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [163]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

markers = gmaps.marker_layer(locations,hotel_info,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))